In [22]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt




plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

%pylab inline
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


In [38]:
# loading HPD violations filtered for 2014 
viol = pd.read_csv('Housing_Maintenance_Code_Violations.csv')
crime = pd.read_csv('NYPD_Complaint_Data_Historic.csv')

/Users/avigailvantu/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
#loading crime data


In [6]:
viol.head(10)

,ViolationID,BuildingID,RegistrationID,BoroID,Borough,HouseNumber,LowHouseNumber,HighHouseNumber,StreetName,StreetCode,...,NovType,ViolationStatus,Latitude,Longitude,CommunityBoard,CouncilDistrict,CensusTract,BIN,BBL,NTA
0,10392273,354043,818372,3,BROOKLYN,1757,1757,1757,PITKIN AVENUE,70930,...,Original,Close,40.670059,-73.908286,16.0,41.0,906.0,3080695.0,3.034890e+09,Brownsville
1,10448977,116971,712919,2,BRONX,2039,2039,2039,VALENTINE AVENUE,70720,...,Original,Close,40.851556,-73.900311,5.0,15.0,381.0,2013554.0,2.031490e+09,Mount Hope
2,10448978,116971,712919,2,BRONX,2039,2039,2039,VALENTINE AVENUE,70720,...,Original,Close,40.851556,-73.900311,5.0,15.0,381.0,2013554.0,2.031490e+09,Mount Hope
3,10451389,116971,712919,2,BRONX,2039,2039,2039,VALENTINE AVENUE,70720,...,Original,Close,40.851556,-73.900311,5.0,15.0,381.0,2013554.0,2.031490e+09,Mount Hope
4,10451447,116971,712919,2,BRONX,2039,2039,2039,VALENTINE AVENUE,70720,...,Original,Close,40.851556,-73.900311,5.0,15.0,381.0,2013554.0,2.031490e+09,Mount Hope
5,10451474,116971,712919,2,BRONX,2039,2039,2039,VALENTINE AVENUE,70720,...,Original,Close,40.851556,-73.900311,5.0,15.0,381.0,2013554.0,2.031490e+09,Mount Hope
6,10451477,116971,712919,2,BRONX,2039,2039,2039,VALENTINE AVENUE,70720,...,Original,Close,40.851556,-73.900311,5.0,15.0,381.0,2013554.0,2.031490e+09,Mount Hope
7,10454776,138349,337388,3,BROOKLYN,5510,5510,5514,13 AVENUE,6380,...,Original,Close,40.631547,-73.995756,12.0,44.0,216.0,3327728.0,3.056830e+09,Borough Park
8,10485688,66501,712955,2,BRONX,500,500,500,EAST 183 STREET,27150,...,Original,Close,40.854914,-73.892404,6.0,15.0,385.0,2011556.0,2.030500e+09,Claremont-Bathgate
9,10485693,66501,712955,2,BRONX,500,500,500,EAST 183 STREET,27150,...,Original,Close,40.854914,-73.892404,6.0,15.0,385.0,2011556.0,2.030500e+09,Claremont-Bathgate


In [7]:
len(viol)

397274

In [41]:
len(crime)

491131

In [42]:
crime.columns

Index([u'CMPLNT_NUM', u'CMPLNT_FR_DT', u'CMPLNT_FR_TM', u'CMPLNT_TO_DT',
       u'CMPLNT_TO_TM', u'RPT_DT', u'KY_CD', u'OFNS_DESC', u'PD_CD',
       u'PD_DESC', u'CRM_ATPT_CPTD_CD', u'LAW_CAT_CD', u'JURIS_DESC',
       u'BORO_NM', u'ADDR_PCT_CD', u'LOC_OF_OCCUR_DESC', u'PREM_TYP_DESC',
       u'PARKS_NM', u'HADEVELOPT', u'X_COORD_CD', u'Y_COORD_CD', u'Latitude',
       u'Longitude', u'Lat_Lon'],
      dtype='object')

In [44]:
crime.LAW_CAT_CD.unique()

array(['MISDEMEANOR', 'FELONY', 'VIOLATION'], dtype=object)

In [45]:
crime.PD_DESC.unique()

array(['FRAUD,UNCLASSIFIED-MISDEMEANOR', 'AGGRAVATED HARASSMENT 2',
       'LARCENY,GRAND BY BANK ACCT COMPROMISE-UNCLASSIFIED',
       'SEXUAL ABUSE 3,2', 'SEXUAL MISCONDUCT,INTERCOURSE', 'SEXUAL ABUSE',
       'HARASSMENT,SUBD 3,4,5', 'COERCION 1', 'RAPE 3',
       'LARCENY,GRAND BY OPEN/COMPROMISE CELL PHONE ACCT',
       'FRAUD,UNCLASSIFIED-FELONY', 'LARCENY,PETIT OF LICENSE PLATE',
       'CRIMINAL MISCHIEF 4TH, GRAFFIT',
       'LARCENY,GRAND BY OPEN CREDIT CARD (NEW ACCT)', 'RAPE 2',
       'MISCHIEF,CRIMINAL,    UNCL 2ND',
       'LARCENY,GRAND BY CREDIT CARD ACCT COMPROMISE-EXISTING ACCT',
       'SODOMY 3', 'COERCION 2',
       'LARCENY,GRAND BY BANK ACCT COMPROMISE-REPRODUCED CHECK',
       'LARCENY,GRAND BY THEFT OF CREDIT CARD',
       'NY STATE LAWS,UNCLASSIFIED MIS', 'RAPE 1',
       'LARCENY,GRAND BY DISHONEST EMP', 'SODOMY 1', 'RAPE 1,ATTEMPT',
       'CRIMINAL MIS 2 & 3', 'SODOMY 2', 'COURSE OF SEXUAL CONDUCT AGAIN',
       'LARCENY,GRAND FROM BUILDING (NON-RESIDENCE)